# Laporan Praktikum 2 – Kecerdasan Komputasional  
## Proof by Resolution, Forward & Backward Chaining, dan First-Order Logic  


### Identitas Mahasiswa
- Nama  : Wayan Raditya Putra  
- NRP   : 5054241029  
- Program Studi : Rekayasa Kecerdasan Artifisial
- Angkatan : 2024  


### Tujuan Praktikum
Praktikum ini bertujuan untuk:  
1. Memahami konsep dasar Proof by Resolution sebagai teknik inferensi dalam logika.  
2. Mempelajari mekanisme Forward Chaining dan Backward Chaining untuk penalaran berbasis aturan.  
3. Mengenal representasi First-Order Logic (FOL) dan bagaimana FOL digunakan dalam basis pengetahuan.  
4. Mengimplementasikan berbagai teknik inferensi untuk menyelesaikan permasalahan berbasis logika komputasional.  


### Deskripsi Singkat
Pada praktikum kedua ini, mahasiswa diminta untuk:  
- Menyusun representasi basis pengetahuan dalam bentuk proposisi maupun logika orde pertama.  
- Menerapkan teknik inferensi seperti resolution, forward chaining, backward chaining.  
- Melakukan percobaan terhadap kasus sederhana untuk melihat bagaimana sistem inferensi menghasilkan kesimpulan dari basis pengetahuan.  


## Instalasi dan Import Library

**Tujuan:**  
Menyiapkan library dan modul yang diperlukan sebelum menjalankan praktikum. Pada tahap ini dilakukan instalasi paket eksternal serta import fungsi-fungsi yang sudah tersedia di file pendukung.

```python
%pip install ipythonblocks
%pip install qpsolvers

from utils import *
from logic import *
from notebook import psource
````
**Penjelasan:**

* `%pip install ipythonblocks` → menginstal library **ipythonblocks** yang digunakan untuk menampilkan blok warna pada notebook.
* `%pip install qpsolvers` → menginstal library **qpsolvers** yang digunakan untuk menyelesaikan masalah Quadratic Programming.
* `from utils import *` → mengimpor seluruh fungsi dari file `utils.py` yang berisi kumpulan fungsi bantu.
* `from logic import *` → mengimpor seluruh fungsi dari file `logic.py` yang berisi implementasi logika proposisional dan inference.
* `from notebook import psource` → mengimpor fungsi `psource` dari `notebook.py`, yang digunakan untuk menampilkan source code suatu fungsi langsung di notebook.



In [150]:
%pip install ipythonblocks
%pip install qpsolvers
from utils import *
from logic import *
from notebook import psource
import inspect

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Inferensi pada Basis Pengetahuan Proposisional

**Tujuan:**  
Pada bagian ini kita akan mempelajari dua teknik utama yang digunakan untuk memeriksa apakah suatu kalimat (sentence) dapat di-*entail* oleh sebuah basis pengetahuan (`KB`).  






### Proof by Resolution

Tujuan kita adalah memeriksa apakah $KB \vDash \alpha$, yaitu apakah $KB \implies \alpha$ bernilai benar di setiap model.  

Contoh: kita ingin memeriksa apakah $P \implies Q$ valid.  
Untuk itu kita periksa keterpuasan dari $\neg (P \implies Q)$, yang bisa ditulis ulang menjadi:  

$$
P \land \neg Q
$$

Jika ekspresi $P \land \neg Q$ **tidak terpuaskan (unsatisfiable)**, maka $P \implies Q$ pasti benar di semua model.  


### Konsep Dasar

Teknik ini dikenal sebagai **pembuktian dengan kontradiksi** (*proof by contradiction*).  
Langkahnya: kita mengasumsikan bahwa $\alpha$ salah, lalu menunjukkan bahwa asumsi ini menimbulkan kontradiksi dengan fakta-fakta dalam $KB$.  


### Aturan Resolusi

Aturan resolusi berbunyi:  

$$
(l_1 \lor \dots \lor l_k), \quad (m_1 \lor \dots \lor m_n), \quad (l_i \equiv \neg m_j) 
\;\;\;\Rightarrow\;\;\;
(l_1 \lor \dots \lor l_{i-1} \lor l_{i+1} \lor \dots \lor l_k \lor m_1 \lor \dots \lor m_{j-1} \lor m_{j+1} \lor \dots \lor m_n)
$$


### Hasil Resolusi

Proses resolusi dijalankan berulang sampai salah satu kondisi berikut:

1. Tidak ada klausa baru yang bisa ditambahkan → artinya $KB \nvDash \alpha$.  
2. Ditemukan **klausa kosong** → artinya $KB \vDash \alpha$.  

Klausa kosong setara dengan **False**, karena hanya bisa muncul dari resolusi dua klausa saling berlawanan, misalnya $P$ dan $\neg P$.


# Konversi Kalimat ke Bentuk Normal Konjungtif (CNF)

Dalam pembuktian dengan **resolusi**, algoritma tidak bisa langsung menangani kalimat logika yang rumit.  
Karena itu, kalimat yang mengandung **implikasi (→)** dan **bi-implikasi (↔)** harus disederhanakan dulu.

👉 Fakta penting: **setiap kalimat logika proposisional setara dengan bentuk konjungsi dari klausa.**  
Bentuk ini disebut **Conjunctive Normal Form (CNF)**, yaitu **konjungsi (AND)** dari beberapa **disjungsi (OR) literal**.

Contoh CNF:

$$
(A \lor B) \land (\neg B \lor C \lor \neg D) \land (D \lor \neg E)
$$

Bentuk ini sama seperti **Product of Sums (POS)** di elektronika digital.


## Tahapan Konversi ke CNF

### 1. Ubah Bi-Implikasi ke Implikasi
$$
\alpha \iff \beta \equiv (\alpha \implies \beta) \land (\beta \implies \alpha)
$$

Jika bagian kanan berupa kalimat majemuk:
$$
\alpha \iff (\beta \lor \gamma) \equiv (\alpha \implies (\beta \lor \gamma)) \land ((\beta \lor \gamma) \implies \alpha)
$$


### 2. Ubah Implikasi ke Bentuk Setara
$$
\alpha \implies \beta \equiv \neg \alpha \lor \beta
$$


### 3. Pindahkan Negasi ke Literal
Negasi hanya boleh menempel pada proposisi atomik.  
Gunakan **Hukum De Morgan**:

$$
\neg(\alpha \land \beta) \equiv (\neg \alpha \lor \neg \beta)
$$

$$
\neg(\alpha \lor \beta) \equiv (\neg \alpha \land \neg \beta)
$$


### 4. Distribusikan Disjungsi terhadap Konjungsi
Agar sesuai format CNF, lakukan distribusi:

$$
(\alpha \lor (\beta \land \gamma)) \equiv (\alpha \lor \beta) \land (\alpha \lor \gamma)
$$

Bentuk akhir yang kita inginkan:

$$
(\alpha_1 \lor \alpha_2 \lor \dots) \land (\beta_1 \lor \beta_2 \lor \dots) \land (\gamma_1 \lor \gamma_2 \lor \dots)
$$


## Inti Proses
- CNF = **“AND of ORs”** (konjungsi dari disjungsi literal).  
- Proses konversi = hilangkan ↔ dan →, dorong negasi ke literal, lalu distribusikan OR atas AND.  
- Dengan CNF, kalimat siap dipakai dalam algoritma **resolusi**.




In [151]:
print(inspect.getsource(to_cnf))

def to_cnf(s):
    """
    [Page 253]
    Convert a propositional logical sentence to conjunctive normal form.
    That is, to the form ((A | ~B | ...) & (B | C | ...) & ...)
    >>> to_cnf('~(B | C)')
    (~B & ~C)
    """
    s = expr(s)
    if isinstance(s, str):
        s = expr(s)
    s = eliminate_implications(s)  # Steps 1, 2 from p. 253
    s = move_not_inwards(s)  # Step 3
    return distribute_and_over_or(s)  # Step 4



## Fungsi `to_cnf`
Untuk mengubah sebuah kalimat logika proposisional ke bentuk **Conjunctive Normal Form (CNF)**, kita gunakan fungsi `to_cnf`.  

```python
def to_cnf(s): 
    """
    [Page 253]
    Convert a propositional logical sentence to conjunctive normal form.
    That is, to the form ((A | ~B | ...) & (B | C | ...) & ...)
    >>> to_cnf('~(B | C)')
    (~B & ~C)
    """
    s = expr(s)
    if isinstance(s, str):
        s = expr(s)
    s = eliminate_implications(s)  # Steps 1, 2 from p. 253
    s = move_not_inwards(s)        # Step 3
    return distribute_and_over_or(s)  # Step 4
```

### Penjelasan Tahapan

1. **`eliminate_implications(s)`**

   * Menyelesaikan **Langkah 1 dan 2** dari proses konversi.
   * Semua bi-implikasi (\$\alpha \iff \beta\$) diubah menjadi dua implikasi.
   * Semua implikasi (\$\alpha \implies \beta\$) diubah menjadi bentuk setara (\$\neg \alpha \lor \beta\$).

2. **`move_not_inwards(s)`**

   * Menyelesaikan **Langkah 3**.
   * Negasi digeser ke dalam agar hanya menempel pada literal, dengan bantuan hukum De Morgan.
   * Contoh:

     $$
     \neg (A \lor B) \equiv (\neg A \land \neg B)
     $$

3. **`distribute_and_over_or(s)`**

   * Menyelesaikan **Langkah 4**.
   * Melakukan distribusi disjungsi terhadap konjungsi, sehingga hasil akhir berbentuk **konjungsi dari disjungsi literal**.
   * Contoh:

     $$
     (A \lor (B \land C)) \equiv (A \lor B) \land (A \lor C)
     $$


### Intinya

* Fungsi `to_cnf` = implementasi praktis dari 4 langkah konversi CNF.
* Setiap kalimat proposisional bisa dipaksa ke CNF → siap dipakai untuk **algoritma resolusi**.



In [152]:
print(inspect.getsource(eliminate_implications))
print(inspect.getsource(move_not_inwards))
print(inspect.getsource(distribute_and_over_or))


def eliminate_implications(s):
    """Change implications into equivalent form with only &, |, and ~ as logical operators."""
    s = expr(s)
    if not s.args or is_symbol(s.op):
        return s  # Atoms are unchanged.
    args = list(map(eliminate_implications, s.args))
    a, b = args[0], args[-1]
    if s.op == '==>':
        return b | ~a
    elif s.op == '<==':
        return a | ~b
    elif s.op == '<=>':
        return (a | ~b) & (b | ~a)
    elif s.op == '^':
        assert len(args) == 2  # TODO: relax this restriction
        return (a & ~b) | (~a & b)
    else:
        assert s.op in ('&', '|', '~')
        return Expr(s.op, *args)

def move_not_inwards(s):
    """Rewrite sentence s by moving negation sign inward.
    >>> move_not_inwards(~(A | B))
    (~A & ~B)
    """
    s = expr(s)
    if s.op == '~':
        def NOT(b):
            return move_not_inwards(~b)

        a = s.args[0]
        if a.op == '~':
            return move_not_inwards(a.args[0])  # ~~A ==> A
   


## Fungsi Pendukung `to_cnf`

Agar `to_cnf` bekerja, ada tiga fungsi utama yang menjalankan proses sesuai 4 langkah konversi CNF:

### 1. `eliminate_implications(s)`

```python
def eliminate_implications(s):
    """Change implications into equivalent form with only &, |, and ~ as logical operators."""
    s = expr(s)
    if not s.args or is_symbol(s.op):
        return s  # Atoms are unchanged.
    args = list(map(eliminate_implications, s.args))
    a, b = args[0], args[-1]
    if s.op == '==>':
        return b | ~a
    elif s.op == '<==':
        return a | ~b
    elif s.op == '<=>':
        return (a | ~b) & (b | ~a)
    elif s.op == '^':
        assert len(args) == 2
        return (a & ~b) | (~a & b)
    else:
        assert s.op in ('&', '|', '~')
        return Expr(s.op, *args)
```

**Fungsi:**

* Menghilangkan semua **implikasi (⇒)** dan **bi-implikasi (⇔)**.
* Hasil akhirnya hanya mengandung operator `&` (AND), `|` (OR), dan `~` (NOT).
* Contoh:

$$
(P \implies Q) \equiv (\neg P \lor Q)
$$

$$
(P \iff Q) \equiv (P \implies Q) \land (Q \implies P)
$$


### 2. `move_not_inwards(s)`

```python
def move_not_inwards(s):
    """Rewrite sentence s by moving negation sign inward.
    >>> move_not_inwards(~(A | B))
    (~A & ~B)
    """
    s = expr(s)
    if s.op == '~':
        def NOT(b):
            return move_not_inwards(~b)

        a = s.args[0]
        if a.op == '~':
            return move_not_inwards(a.args[0])  # ~~A ==> A
        if a.op == '&':
            return associate('|', list(map(NOT, a.args)))
        if a.op == '|':
            return associate('&', list(map(NOT, a.args)))
        return s
    elif is_symbol(s.op) or not s.args:
        return s
    else:
        return Expr(s.op, *list(map(move_not_inwards, s.args)))
```

**Fungsi:**

* Memindahkan **negasi** supaya hanya menempel pada literal (variabel atomik).
* Menggunakan **Hukum De Morgan** dan penyederhanaan ganda:

$$
\neg (A \lor B) \equiv (\neg A \land \neg B)
$$

$$
\neg (A \land B) \equiv (\neg A \lor \neg B)
$$

$$
\neg(\neg A) \equiv A
$$


### 3. `distribute_and_over_or(s)`

```python
def distribute_and_over_or(s):
    """Given a sentence s consisting of conjunctions and disjunctions
    of literals, return an equivalent sentence in CNF.
    >>> distribute_and_over_or((A & B) | C)
    ((A | C) & (B | C))
    """
    s = expr(s)
    if s.op == '|':
        s = associate('|', s.args)
        if s.op != '|':
            return distribute_and_over_or(s)
        if len(s.args) == 0:
            return False
        if len(s.args) == 1:
            return distribute_and_over_or(s.args[0])
        conj = first(arg for arg in s.args if arg.op == '&')
        if not conj:
            return s
        others = [a for a in s.args if a is not conj]
        rest = associate('|', others)
        return associate('&', [distribute_and_over_or(c | rest)
                               for c in conj.args])
    elif s.op == '&':
        return associate('&', list(map(distribute_and_over_or, s.args)))
    else:
        return s
```

**Fungsi:**

* Melakukan distribusi **OR terhadap AND**, supaya hasil akhir sesuai format CNF.
* Contoh:

$$
(A \land B) \lor C \equiv (A \lor C) \land (B \lor C)
$$


## Alur Lengkap

Jadi, `to_cnf` menjalankan ketiga fungsi ini berurutan:

1. **Hilangkan implikasi** → `eliminate_implications(s)`
2. **Pindahkan negasi** → `move_not_inwards(s)`
3. **Distribusikan OR atas AND** → `distribute_and_over_or(s)`

Hasil akhirnya selalu berbentuk:

$$
(\alpha_1 \lor \alpha_2 \lor \dots) \land (\beta_1 \lor \beta_2 \lor \dots) \land \dots
$$

siap dipakai dalam algoritma **resolusi**.




Mari kita coba untuk konversi beberapa kalimat menjadi cnf

In [153]:
A, B, C, D = expr('A, B, C, D')
to_cnf(A |'<=>'| B)

((A | ~B) & (B | ~A))

In [154]:
to_cnf(A |'<=>'| (B & C))

((A | ~B | ~C) & (B | ~A) & (C | ~A))

In [155]:
to_cnf(A & (B | (C & D)))

(A & (C | B) & (D | B))

In [156]:
to_cnf((A |'<=>'| ~B) |'==>'| (C | ~D))

((B | ~A | C | ~D) & (A | ~A | C | ~D) & (B | ~B | C | ~D) & (A | ~B | C | ~D))

## Implementasi `pl_resolution`

Setelah memahami fungsi `to_cnf`, kita bisa lihat bagaimana fungsi ini digunakan dalam algoritma **Proof by Resolution**.  
Berikut kode dari fungsi `pl_resolution`:

```python
def pl_resolution(kb, alpha):
    """
    [Figure 7.12]
    Propositional-logic resolution: say if alpha follows from KB.
    >>> pl_resolution(horn_clauses_KB, A)
    True
    """
    clauses = kb.clauses + conjuncts(to_cnf(~alpha))
    new = set()
    while True:
        n = len(clauses)
        pairs = [(clauses[i], clauses[j])
                 for i in range(n) for j in range(i + 1, n)]
        for (ci, cj) in pairs:
            resolvents = pl_resolve(ci, cj)
            if False in resolvents:
                return True
            new = new.union(set(resolvents))
        if new.issubset(set(clauses)):
            return False
        for c in new:
            if c not in clauses:
                clauses.append(c)
```

### Alur Kerja `pl_resolution`

1. **Persiapan klausa**

   * Semua klausa dalam basis pengetahuan (`kb.clauses`) digabung dengan hasil konversi CNF dari negasi query `~alpha`.
   * Hal ini sesuai prinsip resolusi: untuk membuktikan $\text{KB} \vDash \alpha$, kita cek apakah $\text{KB} \land \neg \alpha$ tidak terpuaskan.

2. **Inisialisasi**

   * Variabel `new` dibuat sebagai himpunan kosong untuk menampung resolvent (klausa hasil resolusi).

3. **Loop utama**

   * Semua pasangan klausa dalam `clauses` diambil.
   * Untuk setiap pasangan `(ci, cj)`, dijalankan fungsi `pl_resolve` untuk mencari resolvent.

4. **Pengecekan kontradiksi**

   * Jika ada resolvent yang menghasilkan `False` (klausa kosong), berarti kontradiksi ditemukan → $\text{KB} \vDash \alpha$.
   * Fungsi langsung mengembalikan `True`.

5. **Update klausa**

   * Semua resolvent baru dimasukkan ke dalam `new`.
   * Jika `new` hanyalah subset dari klausa lama (tidak ada klausa baru yang muncul), artinya tidak ada resolusi lebih lanjut → algoritma berhenti dengan hasil `False` (tidak terbukti).
   * Jika ada klausa baru, tambahkan ke `clauses` lalu ulangi proses.

### Intuisi

* **Kunci resolusi:** cari pasangan klausa yang punya literal saling berlawanan, lalu gabungkan sisanya.
* **Tujuan:** terus menghasilkan klausa baru sampai:

  1. Ketemu **klausa kosong** (kontradiksi → query terbukti).
  2. Tidak ada klausa baru lagi (tidak terbukti).

Dengan demikian, `pl_resolution` bekerja sebagai prosedur sistematis untuk mengecek entailment menggunakan prinsip **proof by contradiction** dengan bantuan konversi ke **CNF**.


In [157]:
print(inspect.getsource(pl_resolution))


def pl_resolution(kb, alpha):
    """
    [Figure 7.12]
    Propositional-logic resolution: say if alpha follows from KB.
    >>> pl_resolution(horn_clauses_KB, A)
    True
    """
    clauses = kb.clauses + conjuncts(to_cnf(~alpha))
    new = set()
    while True:
        n = len(clauses)
        pairs = [(clauses[i], clauses[j])
                 for i in range(n) for j in range(i + 1, n)]
        for (ci, cj) in pairs:
            resolvents = pl_resolve(ci, cj)
            if False in resolvents:
                return True
            new = new.union(set(resolvents))
        if new.issubset(set(clauses)):
            return False
        for c in new:
            if c not in clauses:
                clauses.append(c)



In [158]:
pl_resolution(wumpus_kb, ~P11), pl_resolution(wumpus_kb, P11)

(True, False)

In [159]:
pl_resolution(wumpus_kb, ~P22), pl_resolution(wumpus_kb, P22)


(False, False)

## Hasil Uji `pl_resolution` pada Wumpus KB

Kita menguji dua query berbeda pada basis pengetahuan Wumpus menggunakan fungsi `pl_resolution`.

### 1. Query pada Kotak (1,1)

```python
pl_resolution(wumpus_kb, ~P11), pl_resolution(wumpus_kb, P11)
# Hasil: (True, False)
```

**Penjelasan:**
* `pl_resolution(wumpus_kb, ~P11) → True`
  Artinya basis pengetahuan **dapat membuktikan bahwa di kotak (1,1) tidak ada pit**.
  Hal ini sesuai aturan awal permainan Wumpus World: posisi awal pemain (1,1) selalu aman.

* `pl_resolution(wumpus_kb, P11) → False`
  Basis pengetahuan **tidak bisa membuktikan bahwa ada pit di (1,1)**, karena memang informasi dalam KB justru menyatakan sebaliknya.


### 2. Query pada Kotak (2,2)

```python
pl_resolution(wumpus_kb, ~P22), pl_resolution(wumpus_kb, P22)
# Hasil: (False, False)
```

**Penjelasan:**

* `pl_resolution(wumpus_kb, ~P22) → False`
  Basis pengetahuan **tidak bisa membuktikan bahwa (2,2) bebas pit**.
  Informasi yang ada belum cukup untuk menyimpulkan aman atau tidak.

* `pl_resolution(wumpus_kb, P22) → False`
  Basis pengetahuan **juga tidak bisa membuktikan bahwa ada pit di (2,2)**.

Dengan kata lain, posisi (2,2) masih **tidak diketahui statusnya** berdasarkan informasi yang tersedia.

### Kesimpulan

* Pada kotak awal (1,1), KB dengan jelas **meng-entail** bahwa tidak ada pit.
* Pada kotak (2,2), KB belum punya informasi yang cukup, sehingga baik `P22` maupun `~P22` tidak bisa dibuktikan.



### Forward dan Backward Chaining

Sebelumnya kita sudah membahas dua algoritma untuk mengecek apakah sebuah kalimat di-*entail* oleh basis pengetahuan (`KB`).  
Di sini kita perkenalkan **algoritma ketiga**, yaitu *forward chaining* dan *backward chaining*.  

Perbedaan utamanya: tujuan sekarang adalah menentukan apakah basis pengetahuan yang hanya berisi **definite clause** dapat meng-entail sebuah simbol proposisi tunggal $q$ (query).  
Namun, ada satu syarat penting: **basis pengetahuan hanya boleh berisi Horn Clauses**.

#### Horn Clauses

- **Horn Clause** adalah disjungsi (OR) dari beberapa literal dengan **paling banyak satu literal positif**.  
- Jika sebuah Horn Clause memiliki tepat satu literal positif, maka ia disebut **Definite Clause**.  
- Jika Horn Clause hanya berisi literal positif tunggal, ia disebut **Fact**.  

Contoh Horn Clause:  

$$
\neg a \lor \neg b \lor \neg c \lor z
$$

Dengan menggunakan Hukum De Morgan, bentuk ini bisa ditulis ulang menjadi:  

$$
a \land b \land c \;\;\implies\;\; z
$$

Artinya: jika $a$, $b$, dan $c$ semuanya benar, maka $z$ juga dapat disimpulkan benar.  
Bentuk ini mirip dengan cara manusia memproses fakta dan pengetahuan untuk menghasilkan kesimpulan baru.

#### Keunggulan Horn Clauses

1. **Definite Clause dapat ditulis sebagai implikasi**  
   - Premis (body) berupa konjungsi literal positif.  
   - Kesimpulan (head) berupa satu literal positif.  
   - Contoh:  

     $$
     a \land b \implies z
     $$  

   - Sebuah literal positif tunggal disebut **fact**.  
   - Dengan bentuk seperti ini, aturan lebih mudah dipahami maupun diimplementasikan.

2. **Bisa digunakan pada Forward dan Backward Chaining**  
   - *Forward chaining* bekerja dengan menambahkan fakta baru secara progresif.  
   - *Backward chaining* bekerja dengan menelusuri mundur dari query menuju fakta dasar.  
   - Kedua algoritma ini berjalan efisien pada Horn Clauses karena bentuknya sudah sangat terstruktur.

3. **Kompleksitas Linear**  
   - Keputusan entailment dengan Horn Clauses hanya memerlukan traversal setiap klausa dalam basis pengetahuan **paling banyak sekali**.  
   - Hal ini membuat proses inferensi jauh lebih efisien dibanding metode enumerasi atau resolusi umum.


#### Implementasi di Kode

Inferensi dengan *forward chaining* dapat diuji menggunakan fungsi:

```python
pl_fc_entails(KB, q)
```

Catatan penting:

* Basis pengetahuan yang dipakai bukanlah instansi `KB` biasa.
* `KB` di sini adalah objek dari kelas **`PropDefiniteKB`**, turunan dari `PropKB` yang dimodifikasi khusus untuk menyimpan **Definite Clause**.
* Perbedaan utama ada pada metode bantu tambahan yang dapat mengembalikan daftar klausa dalam KB yang memiliki simbol tertentu di bagian premis.

Dengan cara ini, forward chaining dapat lebih cepat menemukan aturan mana saja yang relevan ketika sebuah fakta baru muncul.



In [160]:
print(inspect.getsource(PropDefiniteKB.clauses_with_premise))


    def clauses_with_premise(self, p):
        """Return a list of the clauses in KB that have p in their premise.
        This could be cached away for O(1) speed, but we'll recompute it."""
        return [c for c in self.clauses if c.op == '==>' and p in conjuncts(c.args[0])]




## Fungsi `clauses_with_premise`

```python
psource(PropDefiniteKB.clauses_with_premise)
def clauses_with_premise(self, p):
    """Return a list of the clauses in KB that have p in their premise.
    This could be cached away for O(1) speed, but we'll recompute it."""
    return [c for c in self.clauses if c.op == '==>' and p in conjuncts(c.args[0])]
```

## Fungsi

Digunakan untuk mengambil semua klausa dalam Knowledge Base (KB) yang memiliki simbol tertentu `p` pada bagian premis (bagian kiri implikasi).

## Alur Kerja

1. `self.clauses` → daftar semua klausa dalam basis pengetahuan.
2. `c.op == '==>'` → hanya ambil klausa yang berupa implikasi (definite clause).
3. `c.args[0]` → mengambil bagian premis dari klausa (bagian kiri sebelum `==>`).
4. `conjuncts(c.args[0])` → memecah premis menjadi daftar konjungsi.
5. `p in conjuncts(c.args[0])` → mengecek apakah simbol `p` ada di dalam premis.
6. Hasil akhir berupa list semua klausa yang memenuhi kondisi di atas.

## Intuisi

Premis adalah syarat yang harus dipenuhi agar klausa berlaku.
Fungsi ini digunakan untuk mencari aturan yang relevan dengan simbol tertentu di bagian premis.

## Contoh

Misalkan KB berisi klausa:

1. `A & B ==> C`
2. `D ==> E`
3. `B ==> F`

Jika dipanggil:

```python
clauses_with_premise("B")
```

Maka hasilnya adalah:

* `A & B ==> C`
* `B ==> F`

Kedua klausa tersebut dipilih karena premisnya mengandung `B`.

## Fungsi `pl_fc_entails`

```python
print(inspect.getsource(pl_fc_entails))
def pl_fc_entails(kb, q):
    """
    [Figure 7.15]
    Use forward chaining to see if a PropDefiniteKB entails symbol q.
    >>> pl_fc_entails(horn_clauses_KB, expr('Q'))
    True
    """
    count = {c: len(conjuncts(c.args[0])) for c in kb.clauses if c.op == '==>'}
    inferred = defaultdict(bool)
    agenda = [s for s in kb.clauses if is_prop_symbol(s.op)]
    while agenda:
        p = agenda.pop()
        if p == q:
            return True
        if not inferred[p]:
            inferred[p] = True
            for c in kb.clauses_with_premise(p):
                count[c] -= 1
                if count[c] == 0:
                    agenda.append(c.args[1])
    return False
```
## Fungsi

Digunakan untuk melakukan **forward chaining** pada *Propositional Definite Knowledge Base (PropDefiniteKB)*, untuk mengecek apakah basis pengetahuan dapat **menyimpulkan** suatu simbol query `q`.

## Alur Kerja

1. `count` → menghitung berapa banyak premis (konjungsi) yang harus dipenuhi pada tiap klausa implikasi.
2. `inferred` → penanda apakah suatu simbol sudah pernah disimpulkan (`True/False`).
3. `agenda` → daftar simbol fakta awal (proposisi yang langsung diketahui dari KB).
4. **Loop utama**:

   * Ambil simbol `p` dari `agenda`.
   * Jika `p == q` → kembalikan `True` (berarti query berhasil disimpulkan).
   * Jika `p` belum pernah disimpulkan:

     * Tandai `p` sebagai sudah disimpulkan.
     * Cek klausa yang premisnya mengandung `p`.
     * Kurangi `count[c]` (sisa syarat klausa `c`).
     * Jika `count[c] == 0`, artinya semua syarat terpenuhi → tambahkan kesimpulan (`c.args[1]`) ke `agenda`.
5. Jika `agenda` habis tanpa menemukan `q`, maka kembalikan `False`.

## Intuisi

Algoritma ini meniru cara kita **menarik kesimpulan dari fakta-fakta yang ada**.
Kita mulai dengan fakta awal → cek aturan mana yang bisa dipicu → hasil aturan ditambahkan sebagai fakta baru → ulangi terus sampai query ditemukan atau tidak ada fakta baru lagi.

## Contoh

Misalkan KB berisi klausa:

1. `A ==> B`
2. `B & C ==> D`
3. `D ==> E`
4. Fakta awal: `A`, `C`

Jika kita jalankan:

```python
pl_fc_entails(KB, E)
```

Maka prosesnya:

* Dari `A`, disimpulkan `B`.
* Karena `B` dan `C` ada, maka bisa simpulkan `D`.
* Dari `D`, simpulkan `E`.
* Query `E` berhasil ditemukan → hasil `True`.


In [161]:
print(inspect.getsource(pl_fc_entails))


def pl_fc_entails(kb, q):
    """
    [Figure 7.15]
    Use forward chaining to see if a PropDefiniteKB entails symbol q.
    >>> pl_fc_entails(horn_clauses_KB, expr('Q'))
    True
    """
    count = {c: len(conjuncts(c.args[0])) for c in kb.clauses if c.op == '==>'}
    inferred = defaultdict(bool)
    agenda = [s for s in kb.clauses if is_prop_symbol(s.op)]
    while agenda:
        p = agenda.pop()
        if p == q:
            return True
        if not inferred[p]:
            inferred[p] = True
            for c in kb.clauses_with_premise(p):
                count[c] -= 1
                if count[c] == 0:
                    agenda.append(c.args[1])
    return False




## Penjelasan Fungsi `pl_fc_entails`

Fungsi ini menerima sebuah **knowledge base (KB)** berupa instance dari `PropDefiniteKB` dan sebuah query `q` sebagai input. <br><br>

* `count` pada awalnya menyimpan jumlah simbol dalam **premis** dari setiap kalimat dalam knowledge base.
* Fungsi pembantu `conjuncts` digunakan untuk memecah suatu kalimat berdasarkan operator konjungsi (`&`).
* `inferred` diinisialisasi sebagai **defaultdict boolean**.
  Variabel ini digunakan untuk mengecek apakah semua premis dari setiap klausa dalam agenda sudah berhasil disimpulkan.
* `agenda` pada awalnya berisi daftar klausa yang sudah diketahui benar oleh knowledge base.
  Fungsi pembantu `is_prop_symbol` digunakan untuk mengecek apakah simbol yang diberikan adalah simbol logika proposisional yang valid. <br><br>

Selanjutnya kita melakukan iterasi melalui `agenda`, dengan mengambil (pop) sebuah simbol `p` pada setiap iterasi.

* Jika query `q` sama dengan `p`, maka dapat dipastikan bahwa entailment berlaku (**entails = True**).
* Agenda kemudian diproses, dengan mengurangi `count` sebanyak satu untuk setiap implikasi yang memiliki premis `p`.
* Sebuah **konklusi** ditambahkan ke agenda ketika nilai `count` mencapai nol.
  Hal ini berarti semua premis dari implikasi tersebut sudah diketahui benar. <br><br>

`clauses_with_premise` adalah metode penting dari kelas `PropDefiniteKB`.
Metode ini mengembalikan daftar klausa dalam knowledge base yang memiliki simbol `p` pada bagian premisnya. <br><br>

Sekarang setelah kita memahami cara kerja fungsi ini, mari kita lihat beberapa contoh penggunaannya.
Namun sebelum itu, kita perlu mendefinisikan knowledge base terlebih dahulu.
Kita akan mengasumsikan bahwa klausa-klausa berikut diketahui benar.


In [162]:
clauses = ['(B & F)==>E', 
           '(A & E & F)==>G', 
           '(B & C)==>F', 
           '(A & B)==>D', 
           '(E & F)==>H', 
           '(H & I)==>J',
           'A', 
           'B', 
           'C']



Sekarang kita akan tell (menambahkan) informasi ini ke dalam knowledge base kita.

In [163]:
definite_clauses_KB = PropDefiniteKB()
for clause in clauses:
    definite_clauses_KB.tell(expr(clause))

We can now check if our knowledge base entails the following queries.

In [164]:
pl_fc_entails(definite_clauses_KB, expr('G'))

True

In [165]:
pl_fc_entails(definite_clauses_KB, expr('H'))

True

In [166]:
pl_fc_entails(definite_clauses_KB, expr('I'))

False

In [167]:
pl_fc_entails(definite_clauses_KB, expr('J'))

False

Kita punya klausa:

```
1. (B & F)     ==> E
2. (A & E & F) ==> G
3. (B & C)     ==> F
4. (A & B)     ==> D
5. (E & F)     ==> H
6. (H & I)     ==> J
7. A
8. B
9. C
```

Fakta awal = **A, B, C**.
Algoritma **forward chaining (`pl_fc_entails`)** akan menambahkan fakta baru setiap kali semua premis dari sebuah aturan terpenuhi.


### 1. `pl_fc_entails(..., G)`

* Dari fakta awal `B` dan `C`, aturan (3) aktif → **F**.
* Dari `B` dan `F`, aturan (1) aktif → **E**.
* Sekarang punya `A, B, C, F, E`.
* Aturan (2): `(A & E & F) ==> G`. Semua premis **A, E, F** benar → maka **G** dapat disimpulkan.

**Hasil:** `True` (KB **menyimpulkan G**).

### 2. `pl_fc_entails(..., H)`

* Dari langkah di atas kita sudah punya **E** dan **F**.
* Aturan (5): `(E & F) ==> H`. Semua premis terpenuhi → **H** bisa disimpulkan.

**Hasil:** `True` (KB **menyimpulkan H**).


### 3. `pl_fc_entails(..., I)`

* Coba cek apakah ada aturan yang bisa menghasilkan **I**.
* Tidak ada aturan dengan konsekuen `I`, dan **I** juga bukan fakta awal.

**Hasil:** `False` (KB **tidak bisa menyimpulkan I**).


### 4. `pl_fc_entails(..., J)`

* Aturan (6): `(H & I) ==> J`.
* Kita sudah punya **H** (benar), tapi **I** tidak pernah bisa dibuktikan (lihat poin 3).
* Karena salah satu premis gagal, aturan (6) tidak bisa memicu → **J** tidak dapat disimpulkan.

**Hasil:** `False` (KB **tidak bisa menyimpulkan J**).

### Ringkasan Hasil

| Query | Hasil | Alasan                                                       |
| ----- | ----- | ------------------------------------------------------------ |
| G     | True  | Bisa diturunkan dari A, B, C melalui aturan (3) → (1) → (2). |
| H     | True  | Bisa diturunkan dari E dan F melalui aturan (5).             |
| I     | False | Tidak ada aturan/fakta yang memberi I.                       |
| J     | False | Butuh H dan I, tapi I tidak terbukti → J gagal diturunkan.   |



## Knowledge Base First-Order Logic: FolKB

Kelas FolKB dapat digunakan untuk merepresentasikan knowledge base yang berisi kalimat-kalimat First-order logic.
Cara inisialisasi dan penggunaannya sama seperti PropKB, hanya saja klausa yang digunakan adalah klausa definit first-order.

Pada bagian selanjutnya, kita akan melihat bagaimana cara menuliskan klausa-klausa tersebut untuk membuat sebuah database dan melakukan query terhadapnya.



## Criminal KB

Pada bagian ini kita akan membuat sebuah `FolKB` berdasarkan paragraf berikut: <br/>

*“Hukum menyatakan bahwa merupakan tindak kejahatan bagi seorang warga Amerika untuk menjual senjata kepada negara-negara yang bermusuhan. Negara Nono, musuh Amerika, memiliki beberapa misil, dan semua misil tersebut dijual kepadanya oleh Kolonel West, yang merupakan warga Amerika.”* <br/>

Langkah pertama adalah mengekstrak fakta-fakta dari paragraf tersebut dan mengonversinya menjadi **klausa definit first-order**.

Melakukan ekstraksi fakta dari data secara otomatis memang merupakan tugas yang menantang. Untungnya, kita hanya memiliki sebuah paragraf singkat sehingga ekstraksi dan konversi dapat dilakukan secara manual.

Klausa-klausa tersebut akan disimpan dalam sebuah list yang diberi nama `clauses`.


In [168]:
clauses = []

*“... merupakan tindak kejahatan bagi seorang warga Amerika untuk menjual senjata kepada negara-negara yang bermusuhan”*  
<br/>

Kata kunci yang perlu diperhatikan adalah **'crime'**, **'American'**, **'sell'**, **'weapon'**, dan **'hostile'**.  
Kita gunakan simbol predikat untuk merepresentasikan makna dari kata-kata tersebut:

- `Criminal(x)`: `x` adalah seorang penjahat  
- `American(x)`: `x` adalah warga Amerika  
- `Sells(x, y, z)`: `x` menjual `y` kepada `z`  
- `Weapon(x)`: `x` adalah sebuah senjata  
- `Hostile(x)`: `x` adalah negara bermusuhan  

Sekarang kita gabungkan dengan penamaan variabel yang sesuai untuk menggambarkan arti kalimat tersebut.  
Seorang penjahat `x` adalah juga seorang Amerika `x` yang menjual senjata `y` kepada `z`, di mana `z` adalah sebuah negara bermusuhan.  

$$
\text{American}(x) \land \text{Weapon}(y) \land \text{Sells}(x, y, z) \land \text{Hostile}(z) \implies \text{Criminal}(x)
$$




In [169]:
clauses.append(expr("(American(x) & Weapon(y) & Sells(x, y, z) & Hostile(z)) ==> Criminal(x)"))


*“Negara Nono, musuh Amerika”*  
<br/>

Sekarang kita tahu bahwa **Nono adalah musuh Amerika**.  
Kita merepresentasikan negara-negara ini menggunakan simbol konstanta `Nono` dan `America`.  
Relasi permusuhan ditunjukkan dengan simbol predikat `Enemy`.  

$$
\text{Enemy}(\text{Nono}, \text{America})
$$



In [170]:
clauses.append(expr("Enemy(Nono, America)"))


*“Nono ... memiliki beberapa misil”*  
<br/>

Pernyataan ini menunjukkan **eksistensi sebuah misil** yang dimiliki oleh Nono.  

$$
\exists x \ \text{Owns}(\text{Nono}, x) \land \text{Missile}(x)
$$

Dengan menggunakan **eksistensial instantiation**, kita memperkenalkan sebuah konstanta baru `M1` yang merepresentasikan misil milik Nono.  

$$
\text{Owns}(\text{Nono}, \text{M1}), \quad \text{Missile}(\text{M1})
$$



In [171]:
clauses.append(expr("Owns(Nono, M1)"))
clauses.append(expr("Missile(M1)"))


*“Semua misil milik Nono dijual kepadanya oleh Kolonel West”*  
<br/>

Artinya, jika Nono memiliki sesuatu dan benda tersebut tergolong sebagai misil, maka benda itu dijual kepada Nono oleh West.  

$$
\text{Missile}(x) \land \text{Owns}(\text{Nono}, x) \implies \text{Sells}(\text{West}, x, \text{Nono})
$$



In [172]:
clauses.append(expr("(Missile(x) & Owns(Nono, x)) ==> Sells(West, x, Nono)"))


*“West, yang merupakan orang Amerika”*  
<br/>

Pernyataan ini berarti bahwa **West adalah seorang warga Amerika**.  

$$
\text{American}(\text{West})
$$


In [173]:
clauses.append(expr("American(West)"))

Kita juga tahu, berdasarkan pemahaman bahasa, bahwa **misil adalah senjata** dan bahwa **musuh Amerika termasuk sebagai “hostile”**.  

$$
\text{Missile}(x) \implies \text{Weapon}(x)
$$

$$
\text{Enemy}(x, \text{America}) \implies \text{Hostile}(x)
$$


In [174]:
clauses.append(expr("Missile(x) ==> Weapon(x)"))
clauses.append(expr("Enemy(x, America) ==> Hostile(x)"))

Sekarang setelah kita mengonversi informasi menjadi **klausa definit first-order**,  
kita dapat membuat **knowledge base first-order logic** kita.


In [175]:
crime_kb = FolKB(clauses)

## Fungsi `subst`

```python
def subst(s, x):
    """Substitute the substitution s into the expression x.
    >>> subst({x: 42, y:0}, F(x) + y)
    (F(42) + 0)
    """
    if isinstance(x, list):
        return [subst(s, xi) for xi in x]
    elif isinstance(x, tuple):
        return tuple([subst(s, xi) for xi in x])
    elif not isinstance(x, Expr):
        return x
    elif is_var_symbol(x.op):
        return s.get(x, x)
    else:
        return Expr(x.op, *[subst(s, arg) for arg in x.args])
```

## Fungsi

`subst` adalah fungsi pembantu (*helper function*) yang digunakan untuk **mengganti variabel dengan nilai yang diberikan** dalam pernyataan logika first-order.
Fungsi ini sangat berguna dalam algoritma lanjutan seperti *unification* dan *inference*.

## Alur Kerja

1. Jika `x` berupa **list**, maka lakukan substitusi untuk setiap elemen list.
2. Jika `x` berupa **tuple**, maka lakukan substitusi untuk setiap elemen tuple.
3. Jika `x` **bukan** sebuah ekspresi (`Expr`), kembalikan `x` apa adanya (basis kasus).
4. Jika `x` adalah **variabel**, cek apakah variabel tersebut ada di dalam substitusi `s`:

   * Jika ada, ganti dengan nilai dari `s`.
   * Jika tidak, biarkan tetap `x`.
5. Jika `x` adalah sebuah **ekspresi kompleks**, buat ekspresi baru dengan operator yang sama, lalu lakukan substitusi rekursif pada setiap argumen.

## Contoh

```python
>>> subst({x: 42, y: 0}, F(x) + y)
(F(42) + 0)
```

Artinya: variabel `x` diganti dengan `42` dan `y` diganti dengan `0`.


In [176]:
print(inspect.getsource(subst))


def subst(s, x):
    """Substitute the substitution s into the expression x.
    >>> subst({x: 42, y:0}, F(x) + y)
    (F(42) + 0)
    """
    if isinstance(x, list):
        return [subst(s, xi) for xi in x]
    elif isinstance(x, tuple):
        return tuple([subst(s, xi) for xi in x])
    elif not isinstance(x, Expr):
        return x
    elif is_var_symbol(x.op):
        return s.get(x, x)
    else:
        return Expr(x.op, *[subst(s, arg) for arg in x.args])



Berikut adalah contoh bagaimana fungsi `subst` dapat digunakan.


In [177]:
subst({x: expr('Nono'), y: expr('M1')}, expr('Owns(x, y)'))

Owns(Nono, M1)

## Inferensi dalam First-Order Logic

Pada bagian ini kita akan membahas algoritma **forward chaining** dan **backward chaining** untuk `FolKB`.  
Kedua algoritma tersebut bergantung pada sebuah proses yang disebut **unifikasi (unification)**,  
yang merupakan komponen kunci dari semua algoritma inferensi first-order.


### Unifikasi

Kadang kita perlu mencari **substitusi** yang membuat dua ekspresi logika berbeda terlihat **identik**.  
Proses ini disebut **unifikasi (unification)**, dan dilakukan oleh algoritma `unify`.  

Algoritma ini menerima dua kalimat sebagai input dan mengembalikan sebuah *unifier* jika ada.  
*Unifier* adalah sebuah dictionary yang menyimpan substitusi yang diperlukan agar kedua kalimat menjadi identik.  

Prosesnya dilakukan dengan **merecursive unifikasi komponen** dari sebuah kalimat,  
di mana unifikasi antara simbol variabel `var` dengan simbol konstanta `Const` direpresentasikan sebagai pemetaan:  

$$
\{ \text{var} : \text{Const} \}
$$

Mari kita lihat beberapa contoh.


In [178]:
unify(expr('x'), 3)

{x: 3}

In [179]:
unify(expr('A(x)'), expr('A(B)'))

{x: B}

In [180]:
unify(expr('Cat(x) & Dog(Dobby)'), expr('Cat(Bella) & Dog(y)'))

{x: Bella, y: Dobby}

Dalam kasus di mana **tidak ada substitusi** yang dapat membuat dua kalimat menjadi terunifikasi,  
fungsi akan mengembalikan nilai `None`.


In [181]:
print(unify(expr('Cat(x)'), expr('Dog(Dobby)')))

None


Kita juga perlu berhati-hati agar tidak secara tidak sengaja menggunakan **nama variabel yang sama**.  
Algoritma `unify` akan menganggapnya sebagai satu variabel yang sama,  
sehingga mencegah variabel tersebut untuk memiliki lebih dari satu nilai.


In [182]:
print(unify(expr('Cat(x) & Dog(Dobby)'), expr('Cat(Bella) & Dog(x)')))

None


### Algoritma Forward Chaining

Di sini kita membahas algoritma forward chaining sederhana seperti yang ditunjukkan pada *Figure 9.3*.  
Kita melihat setiap aturan dalam knowledge base dan memeriksa apakah premisnya dapat dipenuhi.  

Proses ini dilakukan dengan mencari **substitusi** yang menyatukan (unify) setiap premis dengan sebuah klausa di `KB`.  
Jika kita dapat melakukan unifikasi pada premis-premis tersebut, maka kesimpulan (dengan substitusi yang sesuai)  
ditambahkan ke dalam `KB`.  

Proses inferensi ini diulangi hingga:  
- Query dapat dijawab, atau  
- Tidak ada lagi kalimat baru yang bisa ditambahkan.  

Setiap kali klausa baru ditambahkan, kita menguji apakah klausa tersebut bisa di-unify dengan query.  
Jika iya, maka substitusi yang dihasilkan dari `unify` menjadi jawaban untuk query.  
Jika kita kehabisan kalimat untuk diinferensikan, artinya query gagal.  

Fungsi `fol_fc_ask` adalah sebuah **generator** yang menghasilkan semua substitusi yang memvalidasi query.  

## Fungsi `fol_fc_ask`

```python
def fol_fc_ask(kb, alpha):
    """
    [Figure 9.3]
    A simple forward-chaining algorithm.
    """
    # TODO: improve efficiency
    kb_consts = list({c for clause in kb.clauses for c in constant_symbols(clause)})

    def enum_subst(p):
        query_vars = list({v for clause in p for v in variables(clause)})
        for assignment_list in itertools.product(kb_consts, repeat=len(query_vars)):
            theta = {x: y for x, y in zip(query_vars, assignment_list)}
            yield theta

    # check if we can answer without new inferences
    for q in kb.clauses:
        phi = unify_mm(q, alpha)
        if phi is not None:
            yield phi

    while True:
        new = []
        for rule in kb.clauses:
            p, q = parse_definite_clause(rule)
            for theta in enum_subst(p):
                ...
        for clause in new:
            kb.tell(clause)
    return None
```

## Penjelasan Fungsi

* **Input:**

  * `kb`: knowledge base berupa `FolKB`.
  * `alpha`: query yang ingin diuji.

* **Langkah kerja:**

  1. **Ambil semua konstanta** dalam `KB` → disimpan di `kb_consts`.
  2. **`enum_subst(p)`**: menghasilkan semua kemungkinan substitusi variabel dengan konstanta dalam `KB`.
  3. **Cek langsung** apakah query bisa dijawab dari fakta yang sudah ada di `KB`. Jika bisa di-*unify*, hasil substitusi langsung dikembalikan.
  4. **Loop inferensi**:

     * Untuk setiap aturan (`rule`) dalam `KB`, pecah jadi premis (`p`) dan kesimpulan (`q`).
     * Cari substitusi (`theta`) yang bisa menyatukan premis dengan fakta yang ada.
     * Jika semua premis terpenuhi, tambahkan kesimpulan baru ke `KB`.
  5. **Ulangi terus** hingga query terjawab atau tidak ada pengetahuan baru yang bisa ditambahkan.

* **Output:**

  * Menghasilkan (*yield*) semua substitusi yang membuat query benar.
  * Jika tidak ada substitusi yang cocok, mengembalikan `None`.



In [183]:
print(inspect.getsource(fol_fc_ask))


def fol_fc_ask(kb, alpha):
    """
    [Figure 9.3]
    A simple forward-chaining algorithm.
    """
    # TODO: improve efficiency
    kb_consts = list({c for clause in kb.clauses for c in constant_symbols(clause)})

    def enum_subst(p):
        query_vars = list({v for clause in p for v in variables(clause)})
        for assignment_list in itertools.product(kb_consts, repeat=len(query_vars)):
            theta = {x: y for x, y in zip(query_vars, assignment_list)}
            yield theta

    # check if we can answer without new inferences
    for q in kb.clauses:
        phi = unify_mm(q, alpha)
        if phi is not None:
            yield phi

    while True:
        new = []
        for rule in kb.clauses:
            p, q = parse_definite_clause(rule)
            for theta in enum_subst(p):
                if set(subst(theta, p)).issubset(set(kb.clauses)):
                    q_ = subst(theta, q)
                    if all([unify_mm(x, q_) is None for x in kb.clauses + new]):
 

Mari kita cari tahu semua negara yang **hostile**.  
Perlu dicatat bahwa kita hanya memberi tahu `KB` bahwa **Nono adalah musuh Amerika**,  
bukan secara langsung bahwa Nono itu **hostile**.


In [184]:
answer = fol_fc_ask(crime_kb, expr('Hostile(x)'))
print(list(answer))

[{x: Nono}]


Generator mengembalikan satu substitusi yang menunjukkan bahwa **Nono adalah sebuah negara hostile**.  
Perhatikan bahwa setelah kita menambahkan satu negara musuh lagi, generator akan mengembalikan **dua substitusi**.


In [185]:
crime_kb.tell(expr('Enemy(JaJa, America)'))
answer = fol_fc_ask(crime_kb, expr('Hostile(x)'))
print(list(answer))

[{x: Nono}, {x: JaJa}]


**Catatan:**  
`fol_fc_ask` melakukan perubahan pada `KB` dengan cara menambahkan kalimat-kalimat baru ke dalamnya.


### Algoritma Backward Chaining

Algoritma ini bekerja **mundur dari goal**, dengan merangkaikan aturan untuk menemukan fakta yang diketahui dan mendukung pembuktian.  

Misalkan `goal` adalah query yang ingin kita cari substitusinya.  
Kita mencari aturan dalam bentuk:

$$
\text{lhs} \implies \text{goal}
$$

di dalam `KB` dan mencoba membuktikan `lhs`.  

Mungkin terdapat beberapa klausa dalam `KB` yang menghasilkan beberapa `lhs`.  
Cukup membuktikan **salah satu** dari klausa tersebut.  

Namun, untuk membuktikan sebuah `lhs`, **semua konjungsi** dalam `lhs` dari klausa tersebut harus dibuktikan.  
Hal ini membuat algoritma backward chaining serupa dengan pencarian **And/Or (And/Or search)**.


#### OR

Bagian *OR* dari algoritma muncul dari pilihan kita untuk memilih **setiap klausa** dalam bentuk:

$$
\text{lhs} \implies \text{goal}
$$

Dengan melihat semua aturan `lhs` yang `rhs`-nya dapat di-*unify* dengan `goal`,  
kita menghasilkan sebuah substitusi yang membuktikan semua konjungsi dalam `lhs`.  

Kita menggunakan fungsi `parse_definite_clause` untuk mendapatkan `lhs` dan `rhs` dari sebuah klausa dalam bentuk:

$$
\text{lhs} \implies \text{rhs}
$$

Untuk fakta atomik, `lhs` berupa **list kosong**.


In [186]:
print(inspect.getsource(fol_bc_or))


def fol_bc_or(kb, goal, theta):
    for rule in kb.fetch_rules_for_goal(goal):
        lhs, rhs = parse_definite_clause(standardize_variables(rule))
        for theta1 in fol_bc_and(kb, lhs, unify_mm(rhs, goal, theta)):
            yield theta1



## Fungsi `fol_bc_or`

```python
def fol_bc_or(kb, goal, theta):
    for rule in kb.fetch_rules_for_goal(goal):
        lhs, rhs = parse_definite_clause(standardize_variables(rule))
        for theta1 in fol_bc_and(kb, lhs, unify_mm(rhs, goal, theta)):
            yield theta1
```


## Fungsi

`fol_bc_or` adalah bagian dari algoritma **backward chaining** yang menangani aspek **OR**.
Artinya, fungsi ini mencoba membuktikan `goal` dengan melihat semua aturan yang bisa menghasilkan `goal` sebagai konsekuensinya.


## Alur Kerja

1. **Ambil aturan relevan:**
   `kb.fetch_rules_for_goal(goal)` mengambil semua aturan dalam `KB` yang memiliki `goal` sebagai bagian dari konsekuensinya (`rhs`).

2. **Standarisasi variabel:**
   `standardize_variables(rule)` memastikan bahwa variabel dalam aturan tidak berbenturan dengan variabel lain di aturan berbeda.

3. **Pisahkan klausa:**
   `parse_definite_clause` memecah aturan menjadi `lhs` (premis) dan `rhs` (konsekuensi).

4. **Unifikasi:**
   `unify_mm(rhs, goal, theta)` mencoba melakukan unifikasi antara `rhs` dengan `goal` menggunakan substitusi awal `theta`.

   * Jika unifikasi berhasil → lanjut ke tahap berikutnya.
   * Jika gagal → aturan tersebut diabaikan.

5. **Panggil `fol_bc_and`:**
   Jika unifikasi berhasil, maka semua premis (`lhs`) dari aturan tersebut harus dibuktikan.
   Fungsi `fol_bc_and` dipanggil untuk mencoba membuktikan semua konjungsi dalam `lhs`.

6. **Hasil substitusi:**
   Untuk setiap substitusi `theta1` yang berhasil membuktikan semua premis, hasil tersebut di-*yield* sebagai kemungkinan jawaban.

## Intuisi

* Fungsi ini melakukan **pencarian bercabang (OR search)**:
  coba semua aturan yang bisa menghasilkan `goal`.
* Jika salah satu aturan berhasil (premis-premisnya terbukti benar), maka `goal` dapat dibuktikan.



#### AND

Bagian *AND* berkaitan dengan pembuktian **semua konjungsi** dalam `lhs`.  
Kita perlu menemukan sebuah **substitusi** yang dapat membuktikan setiap klausa dalam daftar konjungsi tersebut.


In [187]:
print(inspect.getsource(fol_bc_and))


def fol_bc_and(kb, goals, theta):
    if theta is None:
        pass
    elif not goals:
        yield theta
    else:
        first, rest = goals[0], goals[1:]
        for theta1 in fol_bc_or(kb, subst(theta, first), theta):
            for theta2 in fol_bc_and(kb, rest, theta1):
                yield theta2




## Fungsi `fol_bc_and`

```python
def fol_bc_and(kb, goals, theta):
    if theta is None:
        pass
    elif not goals:
        yield theta
    else:
        first, rest = goals[0], goals[1:]
        for theta1 in fol_bc_or(kb, subst(theta, first), theta):
            for theta2 in fol_bc_and(kb, rest, theta1):
                yield theta2
```

## Fungsi

`fol_bc_and` adalah bagian dari algoritma **backward chaining** yang menangani aspek **AND**.
Artinya, fungsi ini mencoba membuktikan **semua premis (lhs)** dalam bentuk konjungsi.

## Alur Kerja

1. **Cek substitusi awal:**

   * Jika `theta` bernilai `None` → berarti unifikasi gagal, maka tidak ada pembuktian yang bisa dilanjutkan.

2. **Cek apakah goals kosong:**

   * Jika `goals` kosong → semua premis sudah terbukti benar, maka `theta` dikembalikan sebagai hasil (yield).

3. **Pisahkan goals:**

   * Ambil klausa pertama `first` dan sisanya `rest`.

4. **Substitusi dan pencarian OR:**

   * Lakukan substitusi pada `first` dengan `theta`.
   * Panggil `fol_bc_or` untuk mencoba membuktikan klausa pertama dengan aturan-aturan yang ada di `KB`.
   * Jika berhasil, akan menghasilkan substitusi baru `theta1`.

5. **Rekursif untuk sisa goals:**

   * Dengan `theta1`, lanjutkan pembuktian `rest` (sisa konjungsi) dengan memanggil `fol_bc_and`.
   * Jika semua berhasil, hasil akhir `theta2` dikembalikan.


## Intuisi

* Fungsi ini memastikan bahwa **semua premis** dari sebuah aturan benar.
* Jika satu saja konjungsi gagal dibuktikan → keseluruhan aturan gagal.
* Karena itu fungsi ini disebut bagian **AND** dari pencarian **And/Or search**.

Sekarang fungsi utama `fol_bc_ask` memanggil `fol_bc_or` dengan substitusi awal berupa **kosong**.  
Metode `ask` dari `FolKB` menggunakan `fol_bc_ask` dan mengambil substitusi pertama yang dikembalikan oleh generator untuk menjawab query.  

Mari kita lakukan query pada knowledge base yang telah kita buat dari `clauses` untuk mencari **negara-negara hostile**.


In [188]:
# Rebuild KB because running fol_fc_ask would add new facts to the KB
crime_kb = FolKB(clauses)

In [189]:
crime_kb.ask(expr('Hostile(x)'))

{v_194: Nono, x: Nono}